<a href="https://colab.research.google.com/github/nefario7/cmu-deeplearning/blob/working-hw1/hw1_pt2_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount drive and download dataset

In [ ]:
from IPython.display import clear_output 
! apt-get install -y -qq software-properties-common python-software-properties module-init-tools
! add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
! apt-get update -qq 2>&1 > /dev/null
! apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

! google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
! echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
% cd /content
! mkdir cmudrive
% cd ..
! google-drive-ocamlfuse /content/cmudrive
! pip install kaggle wandb torch-summary
! mkdir ~/.kaggle
! cp /content/cmudrive/IDL/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! wandb login

! pip install --upgrade --force-reinstall --no-deps kaggle 
! kaggle config set -n path -v /content
! kaggle competitions download -c 11-785-s22-hw1p2
! unzip -q /content/competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip -d /content/hw1-data

clear_output()

### Dependencies

In [ ]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [ ]:
import wandb
import yaml
import time
import csv
import pandas as pd
from torchsummary import summary
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
! pip3 install adamp
from adamp import AdamP

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=9ef9016070eb31b67cc9d3fde03b6f030419ac578727cd9a96adae45a0c1a958
  Stored in directory: /root/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built adamp


### Network Architecture and Dataloaders

In [ ]:
def model_saving(path, args, save_metadata=True, exp="Experiment", ensemble=False):
    save_name = ''
    if not args['CSV_PATH']:
        save_name += "full_"

    for parameter, val in args.items():
        abbr = parameter[0] if len(parameter) > 2 else parameter
        if parameter == 'lr' :
            data = abbr + str(val)
            save_name += data
            break
        else:
            data = abbr + str(val) + '_'
            save_name += data

    if ensemble:
        save_name = save_name + "-ver" + str(np.random.randint(10, 100))

    print("\nModel will be saved as : ", save_name)
    save_path = os.path.join(path, save_name)
    try:
        os.mkdir(save_path)
    except FileExistsError:
            d = input("Model name already exists. Delete existing model? (y/n)")
            if d == 'y':
                import shutil
                shutil.rmtree(save_path)
                os.mkdir(save_path)
            else:
                return None

    with open(os.path.join(save_path, 'model_parameters.yaml'), 'w') as metadata:
        yaml.dump({'Experiment': exp}, metadata, indent=8, default_flow_style=False)
        yaml.dump(args, metadata, indent=4, default_flow_style=False)

    return save_path

def save_model(args, model, model_path, save_best=False):
    if save_best:
        torch.save(model.state_dict(), os.path.join(model_path, "best_model.pth"))
    else:
        torch.save(model.state_dict(), os.path.join(model_path, "model.pth"))
        # torch.save(model, os.path.join(model_path, "model.pth"))
    print("Model saved at : ", model_path)

def initialize_weights(m):
  if isinstance(m, nn.BatchNorm2d):
      nn.init.constant_(m.weight.data, 1)
      nn.init.constant_(m.bias.data, 0)
  elif isinstance(m, nn.Linear):
      nn.init.kaiming_uniform_(m.weight.data)
      nn.init.constant_(m.bias.data, 0)
    

In [ ]:
class Network(torch.nn.Module):
    def __init__(self, arch, context=0, drop=0.1):
        super(Network, self).__init__()
        c = (1 + 2 * context)
        INPUT_SIZE = c * 13
        NUM_CLASSES = 40

        layers = []
        sizes = [INPUT_SIZE] + arch + [NUM_CLASSES]
        
        print("Network Architecture")
        print(f"No. of hidden layers  = {len(sizes) - 2}, Max. Width = {max(sizes)}")
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if sizes[i+1] != NUM_CLASSES:
                layers.append(nn.ReLU())
                layers.append(nn.BatchNorm1d(num_features = sizes[i+1]))
                layers.append(nn.Dropout(drop))
        
        self.classifier = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.classifier(A0)
        return x

In [ ]:
class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
        
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
            
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            self.X, self.Y = X, Y
        else:
            X = np.pad(X, ((context,context), (0,0)), 'constant', constant_values=(0,0))
            self.X, self.Y = X, Y
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            xx = self.X[i:(i + 2*self.context + 1)].flatten()
            yy = self.Y[i]
        return xx, yy

### Training and Testing

In [ ]:
def train(args, model, device, train_samples, optimizer, criterion, epoch):
    model.train()
    scaler = GradScaler()

    if args["log"]:
        wandb.watch(model, criterion, log="all", log_freq=10)
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], num_workers=2, pin_memory=True, shuffle=True)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad(set_to_none=True)
            with autocast():
                output = model(data)
                loss = criterion(output, target)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if batch_idx % args['log_interval'] == 0:
                if args['log']:
                    wandb.log({"Training Loss": loss.item()})
                    
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, criterion, dev_samples):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                val_loss = criterion(output, true_y)
                if args['log']:
                    wandb.log({"Validation Loss": val_loss.item()})
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    return train_accuracy, val_loss

In [ ]:
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using ", device)
    
    # Model
    model = Network(args["arch"], args['context'], args['drop']).to(device)
    summary(model)
    model_path = model_saving(
            path=r'/content/cmudrive/IDL/hw1-models-sub', 
            args=args, 
            save_metadata=True, 
            exp="Cylinder",
            ensemble=args["ensemble"]
            )

    # Weight Initialization
    if args["weight"] is not None:
        print("\nInitializing Weights")
        model.apply(initialize_weights)

    # Optimizer
    if args["optimizer"] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=args['lr'], weight_decay=1e-5)
    elif args["optimizer"] == 'sgdn':
        optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=0.9, nesterov=True)
    elif args["optimizer"] == 'adamp':
        optimizer = AdamP(model.parameters(), lr=args['lr'], betas=(0.9, 0.999), weight_decay=1e-2)

    # Scheduler
    if args["scheduler"] == 'rlrop':
        scheduler = ReduceLROnPlateau(optimizer, 'min')
    if args["scheduler"] == 'exp':
        scheduler = ExponentialLR(optimizer, gamma=0.9)

    # Loss
    criterion = torch.nn.CrossEntropyLoss()

    # Dataloaders
    train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath=args['CSV_PATH'])
    dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")

    if args['log']:
        print("Initializing W&B")
        wandb.init(project="hw1-submission", entity="nefario7", config=args)

    max_acc = 0
    print("\n------------------------------Training-----------------------------")
    for epoch in range(1, args['epoch'] + 1):
        start = time.time()
        train(args, model, device, train_samples, optimizer, criterion, epoch)
        test_acc, val_loss = test(args, model, device, criterion, dev_samples)

        if args["scheduler"] is not None:
            scheduler.step(val_loss)
            if args['log']:
                print(f"Learning Rate: {scheduler.get_last_lr()}")
                wandb.log({"Learning Rate": scheduler.get_last_lr()})

        if args['log']:
            wandb.log({"Accuracy": test_acc * 100})
        print(f'Validation Accuracy = {test_acc * 100}%')

        end = time.time()
        print(f'Time taken = {end - start} secs\n')

        if args['save'] and test_acc > max_acc:
            print("\nSaving best model!")
            max_acc = test_acc
            save_model(args, model, model_path, save_best=True)


    print("\n-------------------------Training Complete!------------------------")

    if args['save']:
        save_model(args, model, model_path)

    if args['log']:
        wandb.finish()

### Run

In [ ]:
# Context 16 - 81.95
# Context 32 - 81.872
# LR 0.01 - 75.47
# AdamP - 82.2

# Finale Arch = [4096/2048, 2048, 2048, 2048, 1024, 512, 256]
# Ensemble Arch 1 = [512, 1024, 2048, 4096, 2048, 1024, 512, 256]

args = {
    '': 'Ensemble2',
    'batch_size': 16384,
    'epoch': 15,
    'context': 32,
    'bn': 'after',
    'weight': None, 
    'optimizer': 'adamp',
    'scheduler': None,
    'drop': 0.25,
    'lr': 0.001,
    'arch': [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024],
    'LIBRI_PATH': '/content/hw1-data/hw1p2_student_data',
    'CSV_PATH': None,
    'log_interval': 200,
    'save' : True,
    'log' : True,
    'ensemble': True
}

torch.cuda.empty_cache()
main(args)

Using  cuda
Network Architecture
No. of hidden layers  = 8, Max. Width = 1024
Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Linear: 2-1                       866,304
|    └─ReLU: 2-2                         --
|    └─BatchNorm1d: 2-3                  2,048
|    └─Dropout: 2-4                      --
|    └─Linear: 2-5                       1,049,600
|    └─ReLU: 2-6                         --
|    └─BatchNorm1d: 2-7                  2,048
|    └─Dropout: 2-8                      --
|    └─Linear: 2-9                       1,049,600
|    └─ReLU: 2-10                        --
|    └─BatchNorm1d: 2-11                 2,048
|    └─Dropout: 2-12                     --
|    └─Linear: 2-13                      1,049,600
|    └─ReLU: 2-14                        --
|    └─BatchNorm1d: 2-15                 2,048
|    └─Dropout: 2-16                     --
|    └─Linear: 2-17                      1,049,600
|    └─ReLU: 2-18                   

wandb: Currently logged in as: nefario7 (use `wandb login --relogin` to force relogin)



------------------------------Training-----------------------------
Train Epoch: 1 [0/25351378 (0%)]	Loss: 3.905677
Train Epoch: 1 [3276800/25351378 (13%)]	Loss: 1.048918
Train Epoch: 1 [6553600/25351378 (26%)]	Loss: 0.929813
Train Epoch: 1 [9830400/25351378 (39%)]	Loss: 0.872737
Train Epoch: 1 [13107200/25351378 (52%)]	Loss: 0.833595
Train Epoch: 1 [16384000/25351378 (65%)]	Loss: 0.805122
Train Epoch: 1 [19660800/25351378 (78%)]	Loss: 0.784250
Train Epoch: 1 [22937600/25351378 (90%)]	Loss: 0.767015
Train Epoch: 1 [0/10839756 (0%)]	Loss: 0.769532
Train Epoch: 1 [3276800/10839756 (30%)]	Loss: 0.747903
Train Epoch: 1 [6553600/10839756 (60%)]	Loss: 0.705713
Train Epoch: 1 [9830400/10839756 (91%)]	Loss: 0.717238
Validation Accuracy = 79.28775078761453%
Time taken = 434.29023122787476 secs


Saving best model!
Model saved at :  /content/cmudrive/IDL/hw1-models-sub/full_Ensemble2_b16384_e15_c32_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81
Train Epoch: 2 [0/25351378 (0%)]	Loss: 0.723113
Tr

Accuracy,▁▃▄▅▆▆▇▇▇▇▇████
Training Loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁
Validation Loss,█▇█▆▆▄▆▅▄▄▄▃▄▄▅▃▂▃▃▂▄▂▃▃▃▂▂▃▂▃▂▂▃▄▃▃▃▁▄▃
Accuracy,84.94954
Training Loss,0.52359
Validation Loss,0.45226


## Sweep
[wandb Sweep](https://docs.wandb.ai/guides/sweeps)

In [ ]:
sweep_config = {
  "name" : "hw1-sweep",
  "method": "bayes", 
  "parameters": {
    "optimizer": {
      "distribution": "categorical", 
      "values": [
        "sgdn", 
        "adam"
      ]
    }, 
    "scheduler": {
      "distribution": "categorical", 
      "values": [
        "rlrop", 
        "exp"
      ]
    }, 
    "bn": {
      "distribution": "categorical", 
      "values": [
        "after", 
        "before"
      ]
    },  
    "epoch": {
      "max": 20, 
      "distribution": "int_uniform", 
      "min": 5
    }, 
    "lr": {
      "max": 0.02, 
      "distribution": "uniform", 
      "min": 0.0005
    }, 
    "batch_size": {
      "max": 131072, 
      "distribution": "int_uniform", 
      "min": 2048
    }, 
    "context": {
      "max": 64, 
      "distribution": "int_uniform", 
      "min": 8
    }
  }
}

sweep_id = wandb.sweep(sweep_config)
def train():
    with wandb.init() as run:
        config = wandb.config
        model = make_model(config)
        for epoch in range(config["epochs"]):
            loss = model.fit()  # your model training code here
            wandb.log({"loss": loss, "epoch": epoch})

count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

## Submission

In [ ]:
import csv, yaml
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from sklearn.metrics import accuracy_score
import os, datetime
import pandas as pd
import numpy as np

from sklearn.ensemble import VotingClassifier

class Network(torch.nn.Module):
    def __init__(self, arch, context=0, drop=0.1):
        super(Network, self).__init__()
        c = (1 + 2 * context)
        INPUT_SIZE = c * 13
        NUM_CLASSES = 40

        layers = []
        sizes = [INPUT_SIZE] + arch + [NUM_CLASSES]
        
        print("Network Architecture")
        print(f"No. of hidden layers  = {len(sizes) - 2}, Max. Width = {max(sizes)}")
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if sizes[i+1] != NUM_CLASSES:
                layers.append(nn.ReLU())
                layers.append(nn.BatchNorm1d(num_features = sizes[i+1]))
                layers.append(nn.Dropout(drop))
        
        self.classifier = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.classifier(A0)
        return x

class SubmissionSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, csv_path, sample=20000, shuffle=False, partition="test-clean"):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.X_names = os.listdir(self.X_dir)

        if csv_path:
            self.X_names = list(pd.read_csv(csv_path).file)
        
        self.length = len(self.X_names)
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)

        X = np.concatenate(X)
        return X

class SubmissionItems(torch.utils.data.Dataset):
    def __init__(self, X, context = 0):   
        self.length  = X.shape[0]
        self.context = context

        if context != 0:
            X = np.pad(X, ((context,context), (0,0)), 'constant', constant_values=(0,0))
        self.X = X
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
        else:
            xx = self.X[i:(i + 2 * self.context + 1)].flatten()
        return xx

class SubmissionInference():
    def __init__(self, data_path, csv_path):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.drive_dir = r'/content/cmudrive/IDL'

        self.data_path = data_path
        self.order_csv_path = csv_path
        self.test_samples = SubmissionSamples(data_path = self.data_path, csv_path=self.order_csv_path)

    def __get_labels(self, imodel, iargs):
        imodel.eval()
        labels = []
        print(f"Context = {iargs['context']} | Batch Size = {iargs['batch_size']} | Arch = {iargs['arch']}")
        with torch.no_grad():
            for i in range(len(self.test_samples)):
                X = self.test_samples[i]
                test_items = SubmissionItems(X, context=iargs['context'])
                test_loader = torch.utils.data.DataLoader(test_items, batch_size=iargs['batch_size'], num_workers=2, pin_memory=True, shuffle=False)

                for data in tqdm(test_loader):
                    data = data.float().to(self.device)              
                    output = imodel(data)
                    y = torch.argmax(output, axis=1)
                    labels.extend(y.tolist())
        return labels

    def __load_model(self, model_name, model_type): 
        meta_path = os.path.join(self.drive_dir,  model_type, model_name, 'model_parameters.yaml')
        with open(meta_path, 'r') as meta:
            args = yaml.safe_load(meta)

        model_path = os.path.join(self.drive_dir, model_type, model_name, 'model.pth')
        model = Network(args["arch"], args['context'], args['drop']).to(self.device)
        # summary(model)
        model.load_state_dict(torch.load(model_path))
        return model, args

    def simple_inference(self, model_name, model_type):
        print("Running inference...")
        self.timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        model, args = self.__load_model(model_name, model_type)
        labels = self.__get_labels(model, args)
        
        return labels

    def ensemble_inference(self, model_names, model_type):
        print("Running ensembled inference...")
        self.timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        prelim_labels = []
        for name in model_names:
            print("\n\n\tModel : ", name)
            model, args = self.__load_model(name, model_type)
            prelim_labels.append(self.__get_labels(model, args))

        accs = [86.146, 85.79, 84.95]
        w = accs / np.sum(accs)

        print("Combining predictions...")
        labels_df = pd.DataFrame(prelim_labels)
        labels_df = labels_df.transpose()
        ensembled_labels = labels_df.mode(axis=1, dropna=False).iloc[:, 0].tolist()
        # ensembled_labels = np.where((df.iloc[:,1] == df.iloc[:, 2]), df.iloc[:, 1], df.iloc[:, 0]).tolist()

        return labels_df, ensembled_labels

    def generate_submission(self, save_path, labels): 
        sub_dir = os.path.join(self.drive_dir, save_path + self.timestamp)
        try:
            os.mkdir(sub_dir)
        except:
            print("Couldn't create folder for submission.csv")
            
        sub_path = os.path.join(sub_dir, 'submission.csv')

        with open(sub_path, 'w') as f:
            csvwrite = csv.writer(f)
            csvwrite.writerow(['id', 'label'])
            for i in range(len(labels)):
                csvwrite.writerow([i, labels[i]])

        print(f"File saved at : {sub_path}")
        return sub_path


In [ ]:
model_name = r'full_Finale_b16384_e25_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81'
models = [
          r'full_Finale_b16384_e25_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81',
          r'full_Ensemble1_b16384_e20_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver48',
          r'full_Ensemble2_b16384_e15_c32_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81'
          ]
model_type = r'hw1-models-sub'
data_path = r'/content/hw1-data/hw1p2_student_data'
csv_path = r'/content/hw1-data/test_order.csv'
sub_path = r'hw1-submission/'

inference = SubmissionInference(data_path, csv_path)

# Simple and Ensemble Inference
# labels = inference.simple_inference(model_name, model_type)
labels_df, labels = inference.ensemble_inference(models, model_type)

submission_path = inference.generate_submission(sub_path, labels)
print(f"Preview of submission.csv")
df = pd.read_csv(submission_path)
df.head(10)

Running ensembled inference...


	Model :  full_Finale_b16384_e25_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81
Network Architecture
No. of hidden layers  = 7, Max. Width = 4096
Context = 16 | Batch Size = 16384 | Arch = [4096, 2048, 2048, 2048, 1024, 512, 256]


100%|██████████| 119/119 [00:13<00:00,  9.13it/s]




	Model :  full_Ensemble1_b16384_e20_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver48
Network Architecture
No. of hidden layers  = 8, Max. Width = 4096
Context = 16 | Batch Size = 16384 | Arch = [512, 1024, 2048, 4096, 2048, 1024, 512, 256]


100%|██████████| 119/119 [00:13<00:00,  8.79it/s]




	Model :  full_Ensemble2_b16384_e15_c32_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81
Network Architecture
No. of hidden layers  = 8, Max. Width = 1024
Context = 32 | Batch Size = 16384 | Arch = [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]


100%|██████████| 119/119 [00:10<00:00, 11.51it/s]


Combining predictions...
File saved at : /content/cmudrive/IDL/hw1-submission/2022-02-13_06-22-00/submission.csv
Preview of submission.csv


,id,label
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
5,5,0.0
6,6,0.0
7,7,0.0
8,8,0.0
9,9,0.0


In [ ]:
print(submission_path)
! kaggle competitions submit -c 11-785-s22-hw1p2 -f $submission_path -m "BlSubmission"

/content/cmudrive/IDL/hw1-submission/2022-02-13_06-22-00/submission.csv
100% 24.2M/24.2M [00:00<00:00, 58.8MB/s]
Successfully submitted to Frame-Level Speech Recognition